In [ ]:
import glob
import os
from models.processor import Subscene, Mask
from models.metadata import TileMetadata, DatasetMetadata
from models.verify_tile_coordinates import *

# Inputs
INPUT_SUBSCENE_DIR = os.getenv("INPUT_SUBSCENE_DIR")
INPUT_MASK_DIR = os.getenv("INPUT_MASK_DIR")
INPUT_CLASSIF_TAGS = os.getenv("INPUT_CLASSIF_TAGS")
SHAPEFILE_DIR = os.getenv("SHAPEFILE_DIR")
TILE_SIZE = os.getenv("TILE_SIZE")
FIRST_N = os.getenv("FIRST_N")
# Outputs
OUTPUT_MASKS_DIR = os.getenv("OUTPUT_MASKS_DIR")
OUTPUT_METADATA_SUBSCENES_DIR = os.getenv("OUTPUT_METADATA_SUBSCENES_DIR")
OUTPUT_METADATA_DIR = os.getenv("OUTPUT_METADATA_DIR")

subscenes = glob.glob(f"{INPUT_SUBSCENE_DIR}/*.npy")[:FIRST_N]
masks = glob.glob(f"{INPUT_MASK_DIR}/*.npy")[:FIRST_N]

dataset_metadata = DatasetMetadata()
dataset_metadata.save(OUTPUT_METADATA_DIR)

for subscene_path, mask_path in zip(subscenes, masks):
    print(subscene_path, mask_path)
    subscene_f = os.path.basename(subscene_path)
    mask_f = os.path.basename(mask_path)

    subscene = Subscene(
        INPUT_SUBSCENE_DIR, 
        subscene_f, 
        INPUT_CLASSIF_TAGS, 
        shapefile_dir=SHAPEFILE_DIR,
        tile_size=TILE_SIZE)
    mask = Mask(INPUT_MASK_DIR, mask_f, TILE_SIZE)

    # Save subscene tiles without geospatial information
    # subscene.save_subscene_tiles(OUTPUT_SUBSCENE_DIR, pixel_size=PIXEL_SIZE, crs=CRS)
    # Save subscene tiles with geospatial information
    subscene.save_subscene_tiles_geo(OUTPUT_SUBSCENE_DIR)
    mask.save_mask(OUTPUT_MASKS_DIR)

    metadata = TileMetadata(subscene, mask)
    metadata.save(OUTPUT_METADATA_SUBSCENES_DIR)


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [2]:
# import os

# OUTPUT_SUBSCENE_DIR = "data/processed/images"
# OUTPUT_MASKS_DIR = "data/processed/masks"
# OUTPUT_METADATA_DIR = "data/processed/metadata"

# Check number of generated image tiles
image_tiles = [f for f in os.listdir(OUTPUT_SUBSCENE_DIR) if f.endswith(".tif")]
print(f"✅ {len(image_tiles)} image tiles saved.")

# Check number of generated mask tiles
mask_tiles = [f for f in os.listdir(OUTPUT_MASKS_DIR) if f.endswith(".npy")]
print(f"✅ {len(mask_tiles)} mask tiles saved.")

# Check metadata
metadata_files = [f for f in os.listdir(OUTPUT_METADATA_DIR) if f.endswith(".json")]
print(f"✅ {len(metadata_files)} metadata files saved.")

# Check if number of tiles matches between images & masks
assert len(image_tiles) == len(mask_tiles), "❌ Mismatch between image and mask tiles!"
# assert len(image_tiles) == len(metadata_files), "❌ Mismatch between images and metadata!"

print("✅ All expected files are generated correctly.")

✅ 40 image tiles saved.
✅ 40 mask tiles saved.
✅ 1 metadata files saved.
✅ All expected files are generated correctly.


2️⃣ Validate Tile Dimensions

Objective: Ensure that each tile has the correct (512, 512, bands) shape.

In [3]:
import numpy as np
import rasterio
import glob

tile_paths = glob.glob(f"{OUTPUT_SUBSCENE_DIR}/*.tif")

for tile_path in tile_paths:
    with rasterio.open(tile_path) as img:
        h, w = img.shape
        bands = img.count
        assert (h, w) == (512, 512), f"❌ Incorrect tile size in {tile_path}: {h}x{w} instead of 512x512"
        assert bands == 13, f"❌ Incorrect number of bands in {tile_path}: {bands} instead of 13"

print("✅ Tile dimensions and band count are correct.")

✅ Tile dimensions and band count are correct.


3️⃣ Validate Geospatial Correctness

Objective: Ensure that geospatial metadata correctly maps to the subscene.

(A) Check CRS & Transform for Each Tile

In [4]:
tile_paths = glob.glob(f"{OUTPUT_SUBSCENE_DIR}/*.tif")

for tile_path in tile_paths:
    with rasterio.open(tile_path) as img:
        assert img.crs is not None, f"❌ CRS missing in {tile_path}"
        assert img.transform is not None, f"❌ Transform missing in {tile_path}"
        print(f"✅ {tile_path} has CRS {img.crs} and correct transform.")

✅ data/processed/images/S2B_MSIL1C_20181104T044939_N0206_R076_T46TBM_20181104T074115_TL_RS512_RE1022_CS0_CE512.tif has CRS EPSG:32646 and correct transform.
✅ data/processed/images/S2A_MSIL1C_20180416T081601_N0206_R121_T34HDH_20180416T115825_TL_RS0_RE512_CS512_CE1022.tif has CRS EPSG:32734 and correct transform.
✅ data/processed/images/S2A_MSIL1C_20180416T081601_N0206_R121_T34HDH_20180416T115825_TL_RS0_RE512_CS0_CE512.tif has CRS EPSG:32734 and correct transform.
✅ data/processed/images/S2A_MSIL1C_20181231T004701_N0207_R102_T54HTH_20181231T021140_TL_RS0_RE512_CS512_CE1022.tif has CRS EPSG:32754 and correct transform.
✅ data/processed/images/S2A_MSIL1C_20181231T004701_N0207_R102_T54HTH_20181231T021140_TL_RS512_RE1022_CS512_CE1022.tif has CRS EPSG:32754 and correct transform.
✅ data/processed/images/S2A_MSIL1C_20180808T094031_N0206_R036_T34UDU_20180808T115347_TL_RS0_RE512_CS0_CE512.tif has CRS EPSG:32634 and correct transform.
✅ data/processed/images/S2A_MSIL1C_20180718T135111_N0206_R024

(B) Check Tile Alignment Using Geopandas

In [5]:
import geopandas as gpd
import rasterio
import glob
import os

tile_paths = glob.glob(f"{OUTPUT_SUBSCENE_DIR}/*.tif")

for tile_path in tile_paths:  # Only check 1 tile for now
    # Get corresponding shapefile based on tile name
    shapefile_name = os.path.basename(tile_path).split("_TL")[0]
    shapefile_path = os.path.join(f"{SHAPEFILE_DIR}/{shapefile_name}/{shapefile_name}.shp")

    # Load shapefile (subscene boundary)
    gdf = gpd.read_file(shapefile_path)

    with rasterio.open(tile_path) as img:
        # Extract tile bounds
        minx, miny, maxx, maxy = img.bounds
        # print("\n🔹 Tile Bounds:", img.bounds)

        # Create a GeoDataFrame for the tile's top-left corner
        tile_top_left = gpd.GeoDataFrame(geometry=gpd.points_from_xy([minx], [maxy]), crs=gdf.crs)
        
        # Print debugging info
        # print("\n🔹 Tile Top-Left Point:")
        # print(tile_top_left)

        # print("\n🔹 Subscene Boundary Polygon:")
        # print(gdf.geometry.iloc[0])

        # Use .covers() to allow points on the boundary
        is_inside = gdf.geometry.iloc[0].covers(tile_top_left.geometry.iloc[0])

        # If still false, apply buffer for floating-point precision
        if not is_inside:
            is_inside = gdf.geometry.iloc[0].buffer(1e-3).contains(tile_top_left.geometry.iloc[0])

        # print("\n🔹 Is Tile Inside Subscene?", is_inside)

        # (Optional) Print CRS to confirm they match
        # print("\n🔹 Tile CRS:", img.crs)
        # print("🔹 Shapefile CRS:", gdf.crs)

        # Raise an error if tile is still outside
        assert is_inside, f"❌ Tile {tile_path} is outside the original subscene!"

print("✅ All tiles are correctly positioned within the subscene.")

✅ All tiles are correctly positioned within the subscene.


4️⃣ Check Metadata Accuracy

Objective: Ensure that metadata files match their corresponding images & masks.

In [6]:
import json

metadata_files = glob.glob(f"{OUTPUT_METADATA_SUBSCENES_DIR}/*.json")

for metadata_path in metadata_files:
    with open(metadata_path, "r") as f:
        metadata = json.load(f)

    # product_id = metadata["id"]
    for tile in metadata['tiles']:
        image_file = f"{OUTPUT_SUBSCENE_DIR}/{tile['id']}.tif"
        mask_file = f"{OUTPUT_MASKS_DIR}/{tile['id']}.npy"
        assert os.path.exists(image_file), f"❌ Missing image tile for {tile['id']}"
        assert os.path.exists(mask_file), f"❌ Missing mask tile for {tile['id']}"

        print(f"✅ Metadata for {tile['id']} is consistent with image and mask tiles.")

✅ Metadata for S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS0_RE512_CS0_CE512 is consistent with image and mask tiles.
✅ Metadata for S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS0_RE512_CS512_CE1022 is consistent with image and mask tiles.
✅ Metadata for S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS512_RE1022_CS0_CE512 is consistent with image and mask tiles.
✅ Metadata for S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS512_RE1022_CS512_CE1022 is consistent with image and mask tiles.
✅ Metadata for S2B_MSIL1C_20180218T130239_N0206_R095_T23KNP_20180218T192559_TL_RS0_RE512_CS0_CE512 is consistent with image and mask tiles.
✅ Metadata for S2B_MSIL1C_20180218T130239_N0206_R095_T23KNP_20180218T192559_TL_RS0_RE512_CS512_CE1022 is consistent with image and mask tiles.
✅ Metadata for S2B_MSIL1C_20180218T130239_N0206_R095_T23KNP_20180218T192559_TL_RS512_RE1022_CS0_CE512 is consistent with image and mask tiles.
✅ 

5️⃣ Verify Cloud Coverage Calculation

Objective: Check if cloud coverage is correctly calculated from the mask.

In [13]:
mask_paths = glob.glob(f"{OUTPUT_MASKS_DIR}/*.npy")

for mask_path in mask_paths:
    mask = np.load(mask_path)
    mask_md_path = mask_path.split("_TL")[0] + ".npy"
    
    cloud_mask = mask[:, :, 1]  # Extract cloud channel
    cloud_coverage = np.sum(cloud_mask) / cloud_mask.size
    
    metadata_path = f"{OUTPUT_METADATA_SUBSCENES_DIR}/{os.path.basename(mask_md_path).replace('.npy', '.json')}"
    print(metadata_path)
    with open(metadata_path, "r") as f:
        metadata = json.load(f)
        for tile in metadata['tiles']:
            if tile['id'] == os.path.basename(mask_path).replace(".npy", ""):
                print(tile['id'], mask_path)
                print(tile['cloud_coverage'], cloud_coverage)
                assert np.isclose(tile["cloud_coverage"], cloud_coverage, atol=1e-4), f"❌ Cloud coverage mismatch in {metadata_path}"
    
    print(f"✅ Cloud coverage correctly computed for {metadata_path}.")

data/processed/metadata/subscenes/S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350.json
S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS512_RE1022_CS0_CE512 data/processed/masks/S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS512_RE1022_CS0_CE512.npy
0.2558326721191406 0.2558326721191406
✅ Cloud coverage correctly computed for data/processed/metadata/subscenes/S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350.json.
data/processed/metadata/subscenes/S2B_MSIL1C_20180926T110029_N0206_R094_T33WWV_20180926T152251.json
S2B_MSIL1C_20180926T110029_N0206_R094_T33WWV_20180926T152251_TL_RS512_RE1022_CS512_CE1022 data/processed/masks/S2B_MSIL1C_20180926T110029_N0206_R094_T33WWV_20180926T152251_TL_RS512_RE1022_CS512_CE1022.npy
0.9922027587890625 0.9922027587890625
✅ Cloud coverage correctly computed for data/processed/metadata/subscenes/S2B_MSIL1C_20180926T110029_N0206_R094_T33WWV_20180926T152251.json.
data/processed/metadata/subscenes/S2A_

5️⃣ Verify If tile's calculated coordinates match true subscene coordinates

In [ ]:
from models.utils import load_image_geo
import rasterio
import numpy as np

tile_path = "data/processed/images/S2A_MSIL1C_20180729T025551_N0206_R032_T49PER_20180729T055945_TL_RS512_RE1022_CS0_CE512.tif"
# load_image_geo()
# 

# read single band from rasterio
with rasterio.open(tile_path) as img:
    print(img)
    print(img.profile)
    print(img.shape)

# with rasterio.open(tile_path) as img:
#     img = rasterio.band(img, 1)
#     with rasterio.open()
#     print(img._asdict())
    # print(img.read().shape)#, img.profile

<open DatasetReader name='data/processed/images/S2A_MSIL1C_20180729T025551_N0206_R032_T49PER_20180729T055945_TL_RS512_RE1022_CS0_CE512.tif' mode='r'>
{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 512, 'height': 512, 'count': 13, 'crs': CRS.from_wkt('PROJCS["WGS 84 / UTM zone 49N",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",111],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","32649"]]'), 'transform': Affine(22.544031311154598, 0.0, 581280.0,
       0.0, 22.544031311154598, 1506197.455968689), 'blockxsize': 256, 'blo

ValueError: No indexes to read

In [ ]:
import json
import numpy as np
import glob
import os
import rasterio

# Define paths
TILE_SIZE = (512, 512)

# Load all metadata files
metadata_files = glob.glob(f"{OUTPUT_METADATA_SUBSCENES_DIR}/*.json")

errors = 0  # Counter for mismatches

for metadata_path in metadata_files:
    with open(metadata_path, "r") as f:
        metadata = json.load(f)
    
    # Load original subscene
    subscene_id = metadata["id"]
    subscene_image = os.path.join(INPUT_SUBSCENE_DIR, f"{subscene_id}.npy")
    subscene = np.load(subscene_image).astype(np.uint16)

    for ix, tile in enumerate(metadata['tiles']):
        tile_id = tile["id"]
        original_coords = tile["original_coords"]
        row_start, row_end = original_coords["row_start"], original_coords["row_end"]
        col_start, col_end = original_coords["col_start"], original_coords["col_end"]

        with rasterio.open(f"{OUTPUT_SUBSCENE_DIR}/{tile_id}.tif") as img:
            # Convert from tif to npy
            actual_tile = img.read().astype(np.uint16)
            actual_tile = np.moveaxis(actual_tile, 0, -1)
            actual_tile = actual_tile[:row_end-row_start,
                                      :col_end-col_start,
                                      :]
            
        # Extract the tile from the original subscene using stored coordinates
        subscene_tile_region = subscene[
            row_start:row_end,
            col_start:col_end,
            :
        ]

        if np.array_equal(subscene_tile_region, actual_tile):
            print(f"✅ {tile_id} has correct original coordinates.")
        else:
            print(f"❌ {tile_id} has incorrect original coordinates.")
            errors += 1

        # Validate tile size
        # expected_shape = (TILE_SIZE[0], TILE_SIZE[1], subscene.shape[2])
        if not np.array_equal(subscene_tile_region.shape, actual_tile.shape):
            print(f"❌ Tile size mismatch for {tile_id}:")
            print(f"   Stored Coordinates: row=({row_start}, {row_end}), col=({col_start}, {col_end})")
            print(f"   Expected Shape: {actual_tile.shape}, Actual Shape: {actual_tile.shape}")
            errors += 1

if errors == 0:
    print("✅ All tiles have correctly stored original coordinates!")
else:
    print(f"❌ Found {errors} tiles with incorrect original coordinates!")

0 512 512
0 512 512
✅ S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS0_RE512_CS0_CE512 has correct original coordinates.
0 512 512
512 1022 510
✅ S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS0_RE512_CS512_CE1022 has correct original coordinates.
512 1022 510
0 512 512
✅ S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS512_RE1022_CS0_CE512 has correct original coordinates.
512 1022 510
512 1022 510
✅ S2B_MSIL1C_20180812T135109_N0206_R024_T22MCS_20180812T190350_TL_RS512_RE1022_CS512_CE1022 has correct original coordinates.
0 512 512
0 512 512
✅ S2B_MSIL1C_20180218T130239_N0206_R095_T23KNP_20180218T192559_TL_RS0_RE512_CS0_CE512 has correct original coordinates.
0 512 512
512 1022 510
✅ S2B_MSIL1C_20180218T130239_N0206_R095_T23KNP_20180218T192559_TL_RS0_RE512_CS512_CE1022 has correct original coordinates.
512 1022 510
0 512 512
✅ S2B_MSIL1C_20180218T130239_N0206_R095_T23KNP_20180218T192559_TL_RS512_RE1022_CS0_CE512 has correct origin

In [ ]:
import geopandas as gpd

gdf = gpd.read_file(os.path.join(SHAPEFILE_DIR, "S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600/S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600.shp"))

gdf.geometry.iloc[0].bounds
a = gdf.crs
a.to_string()


In [ ]:
class Tile: 
    def __init__(
        self,
        subscene_filename,
        mask_filename,
        tile_size,
        tile_coords,
    ):
        self.subscene_filename = subscene_filename
        self.mask_filename = mask_filename
        self.tile_size = tile_size
        self.tile_coords = tile_coords

    def load_image(self, filename: str | int, input_dir: str = None) -> NDArray:
        """ Load image from disk """
        if not input_dir:
            input_dir = self.input_dir

        return np.load(f'{input_dir}/{filename}')
    
    def pad(self):
        """ Pad image to make it divisible by the tile size """
        # if not tile_size:
        #     tile_size = self.tile_size

        h, w, _ = image.shape
        pad_h = (tile_size[0] - (h % tile_size[0])) % tile_size[0]
        pad_w = (tile_size[1] - (w % tile_size[1])) % tile_size[1]

        # Zero-padding
        padded_image = np.pad(image,
                              ((0, pad_h), (0, pad_w), (0, 0)),
                              mode='constant', constant_values=0)

        return padded_image, (pad_h, pad_w)




In [ ]:
def calculate_cloud_coverage(mask: NDArray) -> float:
    """
    Calculate cloud coverage percentage from one-hot encoded mask
    
    Args:
        mask: NumPy array of shape (H,W,3) with one-hot encoding
             [CLEAR, CLOUD, CLOUD_SHADOW]
    
    Returns:
        float: Percentage of cloud coverage (0-100)
    """
    # Get only the CLOUD channel (index 1)
    cloud_mask = mask[:, :, 1]
    
    # Calculate percentage
    total_pixels = cloud_mask.size
    cloud_pixels = np.sum(cloud_mask)
    cloud_percentage = (cloud_pixels / total_pixels)
    
    return float(cloud_percentage)

In [ ]:
def generate_metadata(img_filename, mask_filename, tile_obj):
    """ Generate metadata for a tile """
    # Load the image and mask
    tile_obj = {
        "id": "abcde",
        "image_filename": img_filename,
        
    }

In [ ]:
preproc = Preprocessor(in_filepath=SUBSCENE_DIR, out_filepath='data/processed/images')
image = preproc.load_image('S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600.npy')
padded_image, padding = preproc.pad_image(image)
tiles, tile_coords = preproc.tile_image(padded_image, padding)
tile_md = preproc.save_tiles(tiles, tile_coords=tile_coords, out_filepath='data/processed/images')

# Test verification
is_correct = verify_tile_coordinates(image, tiles, tile_coords)
print(f"Tile coordinates are correct: {is_correct}")

print(tile_md)
    

In [ ]:
preproc = Preprocessor(in_filepath=MASKS_DIR, out_filepath='data/processed/masks')
mask = preproc.load_image('S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600.npy')
padded_mask, padding = preproc.pad_image(mask)
mask_tiles, mask_tile_coords = preproc.tile_image(padded_mask, padding)

# Test verification
is_correct = verify_tile_coordinates(mask, mask_tiles, mask_tile_coords)
print(f"Mask tile coordinates are correct: {is_correct}")

In [ ]:
mask[0, 0, :]

In [ ]:
img = np.load(
    f"{SUBSCENE_DIR}/S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600.npy")
mask= np.load(
    f"{MASKS_DIR}/S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600.npy")

img.astype('int16')
mask.astype('int8')

In [ ]:
def read_tif(filepath: str, indexes: int|list[int] = None) -> NDArray:
    """Read GeoTIFF file and return as numpy array in (H,W,C) format"""
    with rasterio.open(filepath) as dataset:
        # Read all bands and stack them in the last dimension
        array = dataset.read(indexes=indexes)  # shape: (C,H,W)
        print(array.shape)
        # Transpose to (H,W,C)
        if (len(array.shape) == 3):
            array = array.transpose(1, 2, 0)
        print(array.shape)
        return array

# Test reading the file we just saved
test_filepath = "./data/transformed/images/test.tif"
loaded_array = read_tif(test_filepath,)

# Verify shape matches original
print(f"Original shape: {tile[0].shape}")
print(f"Loaded shape: {loaded_array.shape}")

# Verify data content matches
print(f"Arrays equal: {np.array_equal(tile[0], loaded_array)}")

In [ ]:
tile

In [ ]:
tile_image(img, (128, 128)).shape

In [ ]:
mask.shape, pad_image(img).shape, 

In [ ]:
mask = np.load(f"{MASKS_DIR}/S2A_MSIL1C_20180101T010721_N0206_R045_T53HLD_20180101T041600.npy")

print(pad_image(mask))